In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
#load database

dataset = pd.read_csv("DiffData_classification_noise.csv")#.dropna()

# print (dataset.head(20)) #head of the data file with 20 rows
# print (dataset.tail(20))
print (dataset.describe().T) #simple statistics like mean, std, percentile

# edit database as needed
# reduce set of importance, rescale values etc...
df = dataset.copy()
df["target"] = [1 if label == "Dgb" else 0 for label in df["Diffusion_type"]]

# setup xdata which is input and y data which is target
#xInput = df.drop(["target"], axis = 1)
xInput = df[["distance","time"]]
yTarget = df[["target"]]

# print(xInput.head(10))
# print(yTarget.head(10))


          count        mean         std       min         25%         50%  \
time      980.0  300.000000  173.647540  0.000000  149.693252  300.000000   
distance  980.0    0.028601    0.016163 -0.000355    0.015133    0.026372   

                 75%         max  
time      450.306748  600.000000  
distance    0.041898    0.064679  


In [3]:
# split data for training and testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(xInput,yTarget, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# from sklearn.svm import SVC
# # Create a baseline SVM classifier
# baseline_svm = SVC()
# baseline_svm.fit(x_train, y_train.values.ravel())
# y_pred = baseline_svm.predict(x_test)

# k nearest neighbours
# from sklearn.neighbors import KNeighborsClassifier
# clf = KNeighborsClassifier(n_neighbors = 3)
# clf.fit(x_train, y_train.values.ravel())

# random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier() 
rf.fit(x_train, y_train.values.ravel())

RandomForestClassifier()

In [4]:
# hyper parameter tuning # random
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint

param_dist = {'n_estimators': randint(50,500), 'max_depth': randint(1,20)}

# create the RandomizedSearchCV object
randomized_search = RandomizedSearchCV(estimator = rf, param_distributions = param_dist, n_iter = 10, cv = 5, verbose = 3)
randomized_search.fit(x_train, y_train.values.ravel())

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ....max_depth=18, n_estimators=409;, score=0.879 total time=   0.3s
[CV 2/5] END ....max_depth=18, n_estimators=409;, score=0.892 total time=   0.4s
[CV 3/5] END ....max_depth=18, n_estimators=409;, score=0.924 total time=   0.3s
[CV 4/5] END ....max_depth=18, n_estimators=409;, score=0.917 total time=   0.3s
[CV 5/5] END ....max_depth=18, n_estimators=409;, score=0.891 total time=   0.7s
[CV 1/5] END ....max_depth=18, n_estimators=384;, score=0.879 total time=   0.4s
[CV 2/5] END ....max_depth=18, n_estimators=384;, score=0.892 total time=   0.4s
[CV 3/5] END ....max_depth=18, n_estimators=384;, score=0.924 total time=   0.2s
[CV 4/5] END ....max_depth=18, n_estimators=384;, score=0.917 total time=   0.2s
[CV 5/5] END ....max_depth=18, n_estimators=384;, score=0.897 total time=   0.9s
[CV 1/5] END .....max_depth=11, n_estimators=58;, score=0.892 total time=   0.1s
[CV 2/5] END .....max_depth=11, n_estimators=58;

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x14639f050>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x145f26d20>},
                   verbose=3)

In [5]:
from sklearn.metrics import accuracy_score 
# get the best hyperparameters and model
best_params_rand = randomized_search.best_params_
best_model_rand = randomized_search.best_estimator_

# evaluate the best model
y_pred_best_rand = best_model_rand.predict(x_test)
accuracy_best_rand = accuracy_score(y_test, y_pred_best_rand)

print(f"Best SVM Accuracy: {accuracy_best_rand:.2f}")
print(f"Best Hyperparameters: {best_params_rand}")

Best SVM Accuracy: 0.90
Best Hyperparameters: {'max_depth': 6, 'n_estimators': 479}


In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report 
print ("\nConfusion Matrix:\n",confusion_matrix(y_test, y_pred_best_rand))
print('\n')
print ("accuracy: ",accuracy_score(y_test, y_pred_best_rand))
print('\n')
print(classification_report(y_test,y_pred_best_rand))


#get CV score
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, x_train, y_train.values.ravel(), scoring='accuracy', cv=5)
print(scores)
print("Final CV Score: {:5.2f}".format(np.mean(scores)))


Confusion Matrix:
 [[86 12]
 [ 7 91]]


accuracy:  0.9030612244897959


              precision    recall  f1-score   support

           0       0.92      0.88      0.90        98
           1       0.88      0.93      0.91        98

    accuracy                           0.90       196
   macro avg       0.90      0.90      0.90       196
weighted avg       0.90      0.90      0.90       196

[0.88535032 0.89808917 0.92993631 0.9044586  0.8974359 ]
Final CV Score:  0.90
